## 29% using a 29M transformer
Steps to reproduce:  
1. Upload this script to google colab or modal
2. (optional) If you want to save checkpoints and results, mount your google drive (colab) / your volume (modal)
3. Click run-all

This script ensures there's no data contamination. 
This produces a `submission.json` file. The `submission.json` file


Notes:
1. The config in this notebook has been tuned for an 80GB A100  
2. Actual results were obtained by running this exact file in 2 phases.  
    - Training on a 40GB A100
    - Take the final checkpoint, and run the inference on an 80GB A100

This will work on smaller GPUs too, but will take longer to train  
For very constrained environments, disable the "do_validate" flag. This avoids checking the validation loss every epoch

In [ ]:
# root_folder, mount_folder = "root", "mnt/mithil-arc" # for modal - REPLACE /mithil-arc WITH YOUR VOLUME NAME
root_folder, mount_folder = "content", "content/drive/MyDrive"  # for colab

%cd /$root_folder/
!git clone https://github.com/mvakde/mdlARC.git # `-b <branch_name> --single-branch` if branch
%cd /$root_folder/mdlARC

In [ ]:
!python dataset_building_scripts/build_datasets.py --datasets arc1 conceptarc  --splits train eval --with-solutions --cleanup none

!rm -rf /$root_folder/mdlARC/interactive-run.ipynb
!rm -rf /$root_folder/mdlARC/clean-env-run.ipynb
!rm -rf /$root_folder/mdlARC/max-clean-env-run.ipynb
!rm -rf /$root_folder/mdlARC/dataset_building_scripts
!rm -rf /$root_folder/mdlARC/readme.md
!rm -rf /$root_folder/mdlARC/img

In [ ]:
from pathlib import Path
import argparse
import importlib
import sys

PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

import utils, tinytransformer, train
importlib.reload(utils)  # pick up code changes during iteration
importlib.reload(tinytransformer)
importlib.reload(train)

args = {
    # run config
    "name": "arc1-refactor",  # download file name
    "GPU": "A100",  # just for logging purposes
    # paths - must pass as Path("<path_to_dir>")
    "data_path": Path("assets/challenges.json"),
    "train_log_file": Path("runs/training_log.txt"),
    "save_path": Path("runs/tiny.pt"),
    "checkpoint_path": None, #Path("runs/tiny.pt"),  # or None to start from scratch
    "checkpoint_epochs": [300,400,500,600,605],  # int N for every N epochs, or list [5, 10, 25]
    
    
    # hyperparameters
    "epochs": 20,
    "batch_size": 32,
    "gradient_accumulation_steps": 1,
    "do_validate": True,
    "val_batch_size": 140,

    "enable_aug": True,
    "max_augments": 10,
    "enable_color_aug": True,
    "color_apply_to_test": True,
    "enable_dihedral_aug": True,
    "dihedral_apply_to_test": True,


    "optimizer": "normuon",  # "adamw" | "normuon"
    "normuon_lr": 1.66e-3,
    "normuon_momentum": 0.95,
    "normuon_beta2": 0.95,
    "lr": 3e-4, #adamw lr

    "warmup_pct": 0.02,
    "wsd_decay_start_pct": 0.8,  # 1.0 = no decay (start at last epoch)
    "lr_floor": 0.0,

    "weight_decay": 0.1,
    "attention_weight_decay": 0.01,
    "token_embedding_weight_decay": 0.01,
    "task_embedding_weight_decay": 0.01,

    "grad_clip": 1.0,
    "dropout": 0.1,
    "seed": 42,

    # Model Architecture
    "d_model": 768,  # 128, 256, 512, 768 | 128, 384, 640
    "n_heads": 12,  # 4, 8, 8/16, 12 | 4, 12, 10
    "d_ff": 3072,  # 512, 1024, 2048, 3072 | 512, 1536, 2560
    "n_layers": 4,  # 4, 6, 16, 16 | 24, 28, 24

    "inference_temperature": None,
    "inference_top_k": None,
}
cfg = argparse.Namespace(**args)

runs_dir = Path("runs")
runs_dir.mkdir(parents=True, exist_ok=True)
with (runs_dir / "config.txt").open("w") as f:
    for k, v in args.items():
        f.write(f"{k}: {v}\n")

model, dataset, dataloader, device, data_path = train.build_model_and_data(cfg)


In [ ]:
# Training only
from time import perf_counter

t_start = perf_counter()
train.train_model(cfg,model=model,dataloader=dataloader,dataset=dataset,device=device,data_path=data_path)
t_duration = perf_counter() - t_start

print(f"Training took {t_duration:.2f}s")
with open(Path("runs/timing.txt"), "w") as f:
    f.write(f"Training: {t_duration:.4f} s\n")

In [ ]:
utils.cleanup_memory(globals()) # cleaning up memory to run inference

In [ ]:
# save data immediately in case eval fails
archive_state = utils.save_run_archive(cfg.name, root_folder, mount_folder, globals_dict=globals())

In [ ]:
from pathlib import Path
import pickle
import importlib
import evaluations
import utils
importlib.reload(evaluations)
importlib.reload(utils)

PATH_BOTH = Path("assets/challenges.json")
# aug_count = max_augments (0 = no aug)

EVAL_CONFIGS = [
    # ("eval_125color_both", 125, PATH_BOTH),
    ("eval_100color_both", 100, PATH_BOTH),
    # ("eval_10color_both", 10, PATH_BOTH),
    # ("eval_0color_both", 0, PATH_BOTH),
    # ("eval_0color_train", 0, PATH_TRAIN)  # Uses TRAIN path (No Geom TTA on Test)
]

EVAL_BATCH_SIZE = 1300
SPLITS = ["test"]
CHECKPOINT_PATH = Path("runs/tiny.pt")
SOLUTIONS_PRESENT = False
EVAL_TASK_IDS = None  # Set to None to evaluate full dataset, or ["00576224", ...] for specific tasks
LOG_CORRECT_GRIDS = False  # Print the actual grid, IDs, and augmentation indices for fully correct grids

eval_results = evaluations.run_evaluation_configs(
    cfg,
    EVAL_CONFIGS,
    eval_batch_size=EVAL_BATCH_SIZE,
    splits=SPLITS,
    checkpoint_path=CHECKPOINT_PATH,
    include_targets=SOLUTIONS_PRESENT,
    task_ids=EVAL_TASK_IDS,
    log_correct_grids=LOG_CORRECT_GRIDS,
)

Path("runs/eval_results.pkl").write_bytes(pickle.dumps(eval_results))
print("Saved runs/eval_results.pkl")


In [ ]:
# refresh Drive zip
archive_state = utils.update_run_archive(cfg.name, root_folder, mount_folder, globals_dict=globals())

In [ ]:
# visualisation
from pathlib import Path
import pickle

if "eval_results" not in globals():
    eval_results = pickle.loads(Path("runs/eval_results.pkl").read_bytes())

if "EVAL_CONFIGS" in globals():
    EVAL_SUB_FOLDER = EVAL_CONFIGS[0][0]
else:
    EVAL_SUB_FOLDER = eval_results[0][0]

VIS_MODE = "!"  # "!" = compare vs solutions, "submission" = attempts-only
utils.visualize_eval_submissions(EVAL_SUB_FOLDER, mode=VIS_MODE, solutions_file="assets/solutions.json")


In [ ]:
# scoring
from pathlib import Path
import pickle

if "eval_results" not in globals():
    eval_results = pickle.loads(Path("runs/eval_results.pkl").read_bytes())

EVAL_SUB_FOLDER = eval_results[0][0]
SOLUTIONS_FILE = Path("assets/solutions.json")
SUBMISSION_FILE = Path(f"runs/{EVAL_SUB_FOLDER}/submission.json")

score = utils.score_arc_submission(SOLUTIONS_FILE, SUBMISSION_FILE)


In [ ]:
# AAIVR flow visualization for one task/pair (augmented inputs + outputs)
from pathlib import Path
import pickle
import aaivr
import utils

if "eval_results" not in globals():
    eval_results = pickle.loads(Path("runs/eval_results.pkl").read_bytes())

if "EVAL_CONFIGS" not in globals():
    raise ValueError("EVAL_CONFIGS not found; rerun eval cell to set configs.")

AAIVR_CONFIG_INDEX = 0  # which eval config to inspect
AAIVR_TASK_ID = None  # set to a task id string to override task index
AAIVR_TASK_INDEX = 0  # 0-based in evaluation pipeline order
AAIVR_INPUT_INDEX = 0  # which test pair (base index before dihedral aug)

eval_entry = eval_results[AAIVR_CONFIG_INDEX]
eval_data = eval_entry[1]
test_results = eval_data.get("test", {}).get("results", [])

eval_config = EVAL_CONFIGS[AAIVR_CONFIG_INDEX]
dataset_path = eval_config[2]
dihedral_enabled = False
color_mappings_by_task = None

aug_ctx = eval_data.get("_aug", {})
if aug_ctx:
    color_mappings_by_task = aug_ctx.get("color_mappings_by_split", {}).get("test")
    dihedral_enabled = aug_ctx.get("dihedral_augmented_by_split", {}).get("test", False)

aaivr.visualize_aaivr_flow(
    test_results,
    dataset_path=dataset_path,
    input_index=AAIVR_INPUT_INDEX,
    task_id=AAIVR_TASK_ID,
    task_index=AAIVR_TASK_INDEX,
    is_dihedral_augmented=dihedral_enabled,
    color_mappings_by_task=color_mappings_by_task,
)
